# Predicting Robot Motion for Bathroom Security

## Problem Summary

The challenge involves predicting the motion of robots moving in a defined space with specific initial positions and velocities. The problem is divided into two parts:

### Part 1: Calculating the Safety Factor
- **Goal**: Simulate robot movement for exactly 100 seconds and calculate the safety factor.
- **Safety Factor**: Determined by counting the number of robots in each quadrant of the space and multiplying these counts.

---



In [ ]:
# Import necessary libraries
import numpy as np

# Define constants
GRID_WIDTH = 101
GRID_HEIGHT = 103
TIME_SECONDS = 100

# Function to parse input
def parse_input(file_path):
    robots = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            px, py = map(int, parts[0][2:].split(','))
            vx, vy = map(int, parts[1][2:].split(','))
            robots.append(((px, py), (vx, vy)))
    return robots

# Function to simulate motion
def simulate_motion(robots, time):
    final_positions = []
    for (px, py), (vx, vy) in robots:
        # Calculate final position after `time` seconds
        final_x = (px + vx * time) % GRID_WIDTH
        final_y = (py + vy * time) % GRID_HEIGHT
        final_positions.append((final_x, final_y))
    return final_positions

# Function to count robots in quadrants
def count_quadrants(positions):
    mid_x = GRID_WIDTH // 2
    mid_y = GRID_HEIGHT // 2
    quadrant_counts = [0, 0, 0, 0]  # Q1, Q2, Q3, Q4

    for x, y in positions:
        if x == mid_x or y == mid_y:
            continue  # Ignore robots on dividing lines
        if x > mid_x and y < mid_y:
            quadrant_counts[0] += 1  # Q1
        elif x < mid_x and y < mid_y:
            quadrant_counts[1] += 1  # Q2
        elif x < mid_x and y > mid_y:
            quadrant_counts[2] += 1  # Q3
        elif x > mid_x and y > mid_y:
            quadrant_counts[3] += 1  # Q4

    return quadrant_counts

# Main function
def main():
    # Parse the input file
    robots = parse_input('input.txt')

    # Simulate motion for 100 seconds
    final_positions = simulate_motion(robots, TIME_SECONDS)

    # Count robots in each quadrant
    quadrant_counts = count_quadrants(final_positions)

    # Calculate the safety factor
    safety_factor = np.prod(quadrant_counts)

    print("Quadrant Counts:", quadrant_counts)
    print("Safety Factor:", safety_factor)

# Run the main function
if __name__ == "__main__":
    main()


### Part 2: Finding the Easter Egg Pattern
- **Goal**: Identify the fewest number of seconds required for the robots to align in a specific recognizable pattern ("Christmas tree").

---


In [ ]:
import numpy as np
from typing import List, Tuple
def parse_input(filename: str) -> List[Tuple[Tuple[int, int], Tuple[int, int]]]:

    """
    Parse input file to extract robot positions and velocities.
    Args:
        filename (str): Path to input file
    Returns:
        List of tuples, each containing (position, velocity)
    """
    robots = []
    with open(filename, 'r') as f:

        for line in f:
            pos_str, vel_str = line.strip().split(' ')
            px, py = map(int, pos_str[2:].split(','))
            vx, vy = map(int, vel_str[2:].split(','))
            robots.append(((px, py), (vx, vy)))

    return robots
def simulate_robots(robots: List[Tuple[Tuple[int, int], Tuple[int, int]]],

                    time: int,
                    width: int,
                    height: int) -> np.ndarray:

    """
    Simulate robot movements and count robot positions after given time.
    Args:
        robots (List): List of robot (position, velocity) tuples
        time (int): Seconds to simulate
        width (int): Space width
        height (int): Space height
    Returns:
        numpy array of robot count per tile
    """
    # Initialize grid to track robot positions

    grid = np.zeros((height, width), dtype=int)
    for (px, py), (vx, vy) in robots:
        # Calculate final position after time, with wraparound
        final_x = (px + vx * time) % width
        final_y = (py + vy * time) % height
        grid[final_y, final_x] += 1
    return grid

def is_christmas_tree(grid: np.ndarray) -> bool:

    """
    Check if the robot positions form a Christmas tree pattern.
    Args:
        grid (numpy array): Robot position grid
    Returns:
        Boolean indicating if a Christmas tree pattern is found
    """

    tree_pattern = [

        [0, 0, 1, 0, 0],   # top of tree
        [0, 1, 1, 1, 0],   # middle of tree
        [1, 1, 1, 1, 1],   # base of tree
    ]

    height, width = grid.shape
    for y in range(height - len(tree_pattern)):
        for x in range(width - len(tree_pattern[0])):
            # Check if this section matches the tree pattern
            match = True
            for dy, row in enumerate(tree_pattern):
                for dx, val in enumerate(row):
                    if val == 1 and grid[y+dy, x+dx] == 0:
                        match = False
                        break
                if not match:
                    break
            if match:
                return True
    return False

def find_christmas_tree_time(filename: str, max_time: int = 10000, width: int = 101, height: int = 103) -> int:

    """
    Find the earliest time when robots form a Christmas tree pattern.
    Args:
        filename (str): Input file path
        max_time (int): Maximum time to search
        width (int): Space width
        height (int): Space height
    Returns:
        Time when Christmas tree pattern is found, or -1 if not found
    """

    # Parse input
    robots = parse_input(filename)
    for time in range(max_time):
        grid = simulate_robots(robots, time, width, height)

        if is_christmas_tree(grid):

            return time

    return -1  # No pattern found within max_time

result = find_christmas_tree_time('input.txt')

print(f"Seconds until Christmas Tree Pattern: {result}")

## Key Details

1. **Input Format**:
   - Each robot's current position and velocity are provided, e.g.,  
     `p=0,4 v=3,-3`  
     - `p=x,y`: Initial position on a 2D grid.  
     - `v=x,y`: Velocity (movement per second in x and y directions).  

2. **Movement Rules**:
   - Robots move in straight lines based on their velocity.
   - If a robot moves beyond the boundaries of the space (width = 101 tiles, height = 103 tiles), it teleports to the opposite side.

3. **Output**:
   - **Part 1**: Number of robots in each quadrant after 100 seconds to calculate the safety factor.  
   - **Part 2**: The smallest time when robots align in the Easter egg pattern.